In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/leekyuyoung20231211/python/main/data/%ED%95%98%EC%9D%B4%ED%8D%BC%ED%8C%8C%EB%9D%BC%EB%A9%94%ED%84%B0%ED%8A%9C%EB%8B%9D.csv')
df.head()

,person_id,Sex,past_login_total,past_1_month_login,past_1_week_login,sub_size,email_type,phone_rat,apple_rat,login
0,0,0,3.0,1.0,1.0,0.0,0,0.000000,0.000000,1
1,1,1,111.0,26.0,7.0,2.0,0,0.072072,0.000000,1
2,3,1,13.0,13.0,11.0,7.0,0,0.076923,1.000000,1
3,4,1,28.0,12.0,5.0,0.0,0,0.071429,0.071429,1
4,5,1,4.0,4.0,4.0,0.0,2,0.000000,0.000000,1


In [ ]:
# person_id: 유저별 고유 아이디
# Sex: 성별
# past_login_total: 과거(5월 8일 이전)에 로그인한 총 횟수
# past_1_month_login: 과거 1달간 로그인한 총 횟수
# past_1_week_login: 과거 1주간 로그인한 총 횟수
# sub_size: 과거에 데이콘 대회에서의 총 제출 수
# email_type: 가입한 이메일 종류
# phone_rat: 폰으로 접속한 비율
# apple_rat: 애플 기기로 접속한 비율
# login: 로그인 여부

- 랜덤포레스트 모델의 하이퍼파라메터를 최적화 최고의 성능을 달성
- Random Seed : 42

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [5]:
# 데이터 분리
X = df.iloc[:, 1:-1]
y = df.iloc[:,-1]

In [6]:
rfc = RandomForestClassifier()
rfc.fit(X,y)
rfc.score(X,y)

0.9633307868601986

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,random_state=42)
rfc.fit(X_train,y_train)
rfc.score(X_train,y_train), rfc.score(X_test,y_test)

(0.9643221202854231, 0.899390243902439)

In [9]:
param_grid = {
    'n_estimators': [50, 100, 200],                    # 트리의 개수
    'max_depth': [None, 10, 20, 30],                    # 각 트리의 최대 깊이
    'min_samples_split': [2, 5, 10],                    # 노드를 분할하기 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4],                      # 리프 노드에 있어야 할 최소 샘플 수
    'max_features': ['sqrt'],                  # 각 분할에서 고려할 최대 특성 수
    'bootstrap': [True, False],                         # 부트스트랩 샘플링 사용 여부
    'class_weight': [None, 'balanced', 'balanced_subsample']  # 클래스 가중치
}
rfc = RandomForestClassifier()
grid =  GridSearchCV(rfc,param_grid=param_grid)
grid.fit(X_train,y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True, False],
                         'class_weight': [None, 'balanced',
                                          'balanced_subsample'],
                         'max_depth': [None, 10, 20, 30],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]})

In [10]:
grid.best_params_, grid.best_score_

({'bootstrap': True,
  'class_weight': None,
  'max_depth': None,
  'max_features': 'sqrt',
  'min_samples_leaf': 4,
  'min_samples_split': 10,
  'n_estimators': 200},
 0.927618356987465)

In [11]:
best_model = grid.best_estimator_
best_model.score(X_train,y_train), best_model.score(X_test,y_test)

(0.9347604485219164, 0.9024390243902439)

하이퍼 파라메터 튜닝 오픈소스 optuna

In [ ]:
!pip install optuna

In [14]:
import optuna

In [28]:
from sklearn.metrics import roc_auc_score, accuracy_score

def objective(trial):
  params = {
    'n_estimators' : trial.suggest_int('n_estimators',50, 200),
    'max_depth' : trial.suggest_int('max_depth',10, 30),
    'min_samples_split' : trial.suggest_int('min_samples_split',2,20),
    'min_samples_leaf' : trial.suggest_int('min_samples_leaf',1,4),
    'max_features' : trial.suggest_categorical('max_features',['sqrt']),
    'bootstrap' : trial.suggest_categorical('bootstrap',[True,False]),
    'class_weight' : trial.suggest_categorical('class_weight',[None, 'balanced', 'balanced_subsample']),
    'random_state' : 42
  }

  # 모델을 초기화 학습
  model = RandomForestClassifier(**params)
  model.fit(X_train,y_train)

  # 검증 데이터에 대한 정확도 계산
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test,y_pred)
  return accuracy

In [ ]:
# optuna 를 사용해서 하이퍼파라메터 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

In [36]:
# 최적의 하이퍼 파라메터 값을 출력
study.best_trial.values, study.best_value, study.best_params

([0.9085365853658537],
 0.9085365853658537,
 {'n_estimators': 110,
  'max_depth': 26,
  'min_samples_split': 17,
  'min_samples_leaf': 2,
  'max_features': 'sqrt',
  'bootstrap': True,
  'class_weight': None})

In [37]:
optuna_best_model = RandomForestClassifier(**study.best_params)
optuna_best_model.fit(X_train,y_train)
optuna_best_model.score(X_train,y_train), optuna_best_model.score(X_test,y_test)

(0.9367991845056065, 0.899390243902439)